In [1]:
!git clone https://github.com/nnminh322/AI_Agent.git
!apt-get update && apt-get -qq install -y nginx
%cd AI_Agent/llm/serving
!pip install uv
!uv pip install -r requirements.txt

Cloning into 'AI_Agent'...
remote: Enumerating objects: 8230, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 8230 (delta 4), reused 13 (delta 3), pack-reused 8214 (from 2)
Receiving objects: 100% (8230/8230), 73.63 MiB | 25.23 MiB/s, done.
Resolving deltas: 100% (732/732), done.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages

In [2]:
import subprocess
import time

command = ["python", "serving.py"]

log_file = open("server.log", "w")

print("Khởi động server vllm...")

process = subprocess.Popen(
    command,
    stdout=log_file,
    stderr=log_file
)

print(f"Server đã khởi động với PID: {process.pid}")
print("Server đang chạy ở chế độ nền. Cell này đã hoàn thành.")

time.sleep(5)

Khởi động server vllm...
Server đã khởi động với PID: 1067
Server đang chạy ở chế độ nền. Cell này đã hoàn thành.


In [3]:
!cat /kaggle/working/AI_Agent/llm/serving/server.log

2025-11-11 08:41:55.383 | INFO     | __main__:run_vllm_server:33 - Khởi chạy server vLLM: /usr/bin/python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen3-4B-Instruct-2507 --tensor-parallel-size 1 --dtype half --max-model-len 4096 --gpu-memory-utilization 0.8 --swap-space 8 --port 8000 --host 127.0.0.1
2025-11-11 08:41:55.384 | INFO     | __main__:run_vllm_server:38 - Đang chờ server tại http://127.0.0.1:8000...
INFO 11-11 08:42:03 [__init__.py:216] Automatically detected platform cuda.
2025-11-11 08:42:05.939157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762850526.173136    1069 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered


In [4]:
# Quay lại thư mục gốc (nơi chứa 'etc' và 'serving')
%cd ..

print("--- 2. Cấu hình Nginx... ---")

# 1. Copy file config từ thư mục "etc" của bạn
# (SỬA LẠI THÀNH "sites-available")
!sudo cp ./etc/nginx/sites-available/llm_gateway.conf /etc/nginx/sites-available/

# 2. Kích hoạt config này (tạo symlink)
!sudo ln -s /etc/nginx/sites-available/llm_gateway.conf /etc/nginx/sites-enabled/

# 3. Xóa file "default" của Nginx để tránh xung đột cổng 80
!sudo rm -f /etc/nginx/sites-enabled/default

# 4. Kiểm tra xem file config của bạn có lỗi không
!sudo nginx -t

# 5. Khởi động Nginx (dùng lệnh trực tiếp, vì Colab không có systemctl)
print("--- 3. Khởi động Nginx... ---")
!sudo nginx

/kaggle/working/AI_Agent/llm
--- 2. Cấu hình Nginx... ---
nginx: the configuration file /etc/nginx/nginx.conf syntax is ok
nginx: configuration file /etc/nginx/nginx.conf test is successful
--- 3. Khởi động Nginx... ---


In [5]:
from pyngrok import ngrok
import getpass

NGROK_AUTH_TOKEN = getpass.getpass("Nhập NGROK_AUTH_TOKEN của bạn: ")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(80, "http")
print(f"API của bạn đang chạy tại: {public_url}")

Nhập NGROK_AUTH_TOKEN của bạn:  ········


API của bạn đang chạy tại: NgrokTunnel: "https://298e4b19873f.ngrok-free.app" -> "http://localhost:80"


In [14]:
!cat /kaggle/working/AI_Agent/llm/serving/server.log

2025-11-11 08:41:55.383 | INFO     | __main__:run_vllm_server:33 - Khởi chạy server vLLM: /usr/bin/python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen3-4B-Instruct-2507 --tensor-parallel-size 1 --dtype half --max-model-len 4096 --gpu-memory-utilization 0.8 --swap-space 8 --port 8000 --host 127.0.0.1
2025-11-11 08:41:55.384 | INFO     | __main__:run_vllm_server:38 - Đang chờ server tại http://127.0.0.1:8000...
INFO 11-11 08:42:03 [__init__.py:216] Automatically detected platform cuda.
2025-11-11 08:42:05.939157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762850526.173136    1069 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762850526.245759    1069 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register